<a href="https://colab.research.google.com/github/upkodah/upkodah-service/blob/Dalci-patch-1/dabang_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from bs4 import BeautifulSoup
import requests
#import json

'''
  1. 서울 전역에 대하여 multi-room 요청을 수행하여 전체 정보를 받아온다.
'''
# 서울 전역에 대한 multi-room 요청(primitive_request) - 나중에 정리
primitive_request_multiroom = "https://www.dabangapp.com/api/3/marker/multi-room?api_version=3.0.1&call_type=web&filters=%7B%22multi_room_type%22%3A%5B0%5D%2C%22selling_type%22%3A%5B0%2C1%5D%2C%22deposit_range%22%3A%5B0%2C999999%5D%2C%22price_range%22%3A%5B0%2C999999%5D%2C%22trade_range%22%3A%5B0%2C999999%5D%2C%22maintenance_cost_range%22%3A%5B0%2C999999%5D%2C%22room_size%22%3A%5B0%2C999999%5D%2C%22supply_space_range%22%3A%5B0%2C999999%5D%2C%22room_floor_multi%22%3A%5B1%2C2%2C3%2C4%2C5%2C6%2C7%2C-1%2C0%5D%2C%22division%22%3Afalse%2C%22duplex%22%3Afalse%2C%22room_type%22%3A%5B1%2C2%5D%2C%22enter_date_range%22%3A%5B0%2C999999%5D%2C%22parking_average_range%22%3A%5B0%2C999999%5D%2C%22household_num_range%22%3A%5B0%2C999999%5D%2C%22parking%22%3Afalse%2C%22animal%22%3Afalse%2C%22short_lease%22%3Afalse%2C%22full_option%22%3Afalse%2C%22built_in%22%3Afalse%2C%22elevator%22%3Afalse%2C%22balcony%22%3Afalse%2C%22loan%22%3Afalse%2C%22safety%22%3Afalse%2C%22pano%22%3Afalse%2C%22deal_type%22%3A%5B0%2C1%5D%7D&location=%5B%5B126.8884562%2C37.4968543%5D%2C%5B127.1093844%2C37.6232711%5D%5D&use_map=kakao&version=1&zoom=7"

# multi-rooom 요청의 json 파일
json_multiroom = requests.get(primitive_request_multiroom).json()

In [30]:
# multi-room JSON 필드 테스트 1 - 키 출력
multiroom_field = list()
for i in json_multiroom:
  multiroom_field.append(i)
print("multi-room fields: ", multiroom_field)

multi-room fields:  ['regions', 'subways', 'univs', 'cctv', 'police', 'safety', 'agents', 'contracts', 'center', 'is_agent_list_zoom']


In [ ]:
# multi-room JSON 필드 테스트 2 - 동별 좌표 목록 출력
# JSON regions 필드의 내용들; dong
regions = json_multiroom['regions']
for regions_content in regions:
  print("name: ", regions_content['name'], ", bbox: ", regions_content['bbox'])

In [61]:
'''
  2. bbox로 지역 단위를 구분하여 저장하고(json parsing), bbox 요청을 수행한다.
  
  url: https://www.dabangapp.com/api/3/room/list/multi-room/bbox?
  params: 딕셔너리를 구성하여 requests.get으로 넘겨 줍니다.
    'api_version':'3.0.1',
    'call_type':'web',
    'filters':bbox_filters_str,
    'page':var_page, 
    'usemap': 'kakao', 
    'version':1, 
    'zoom': 6, 
    'location':var_location

'''
## query string 작성
# 원본 URL
bbox_url = 'https://www.dabangapp.com/api/3/room/list/multi-room/bbox?'

# param: filters
'''
    "multi_room_type":[0],   "selling_type":[0,1], "deposit_range":[0,999999],
    "price_range":[0,999999], "trade_range":[0,999999], "maintenance_cost_range":[0,999999],
    "room_size":[0,999999], "supply_space_range":[0,999999], "room_floor_multi":[1,2,3,4,5,6,7,-,0],
    "division":false, "duplex":false, "room_type":[], "enter_date_range":[0,999999],
    "parking_average_range":[0,999999], "household_num_range":[0,999999],
    "parking":false, "animal":false, "short_lease":false, "full_option":false,
    "built_in":false, "elevator":false, "balcony":false,
    "loan":false, "safety":false, "pano":false, "deal_type":[0,1]}
'''

bbox_filters = []
bbox_filters.append("{\"multi_room_type\":[0], ")                     # 원룸
bbox_filters.append("\"selling_type\":[0, 1], ")                      # 월세, 전세
bbox_filters.append("\"deposit_range\":[0, 999999], ")                # 보증금
bbox_filters.append("\"price_range\":[0, 999999], ")                  # 월세
bbox_filters.append("\"trade_range\":[0, 999999], ")                  # 거래가
bbox_filters.append("\"maintenance_cost_range\":[0, 999999], ")       # 관리비
bbox_filters.append("\"room_size\":[0, 999999], ")                    # 방 크기
bbox_filters.append("\"supply_space_range\":[0, 999999], ")           #
bbox_filters.append("\"room_floor_multi\":[1,2,3,4,5,6,7,-1,0], ")    #
bbox_filters.append("\"division\":false, ")                           #
bbox_filters.append("\"duplex\":false, ")                             # 복층
bbox_filters.append("\"room_type\":[], ")                             #
bbox_filters.append("\"enter_date_range\":[0,999999], ")              # 입주일
bbox_filters.append("\"parking_average_range\":[0,999999], ")         #
bbox_filters.append("\"household_num_range\":[0,999999], ")           # 세대 수
bbox_filters.append("\"parking\":false, ")                            # 주차 가능 여부
bbox_filters.append("\"animal\":false, ")                             # 반려 동물 여부
bbox_filters.append("\"short_lease\":false, ")                        # 단기 대출
bbox_filters.append("\"full_option\":false, ")                        # 풀 옵션
bbox_filters.append("\"built_in\":false, ")                           #
bbox_filters.append("\"elevator\":false, ")                           # 엘레베이터
bbox_filters.append("\"balcony\":false, ")                            # 발코니, 베란다
bbox_filters.append("\"loan\":false, ")                               # 대출
bbox_filters.append("\"safety\":false, ")                             # 보안
bbox_filters.append("\"pano\":false, ")                               # 360도 VR
bbox_filters.append("\"deal_type\":[0,1]}")                           # 거래 유형

bbox_filters_str = "".join(bbox_filters)
# end of filters

'''
 반복문으로 제어가 필요한 query 인수들: page, location
'''
# param: page
var_page = 1                                                          
# param: location
var_location = str(regions[0]['bbox'])                                # bbox의 좌표를 string으로 전달합니다

# params
bbox_params = {'api_version':'3.0.1', 'call_type':'web', 'usemap': 'kakao', 'version':1, 'zoom': 6, 'filters':bbox_filters_str, 'page':var_page, 'location':var_location}
## query string 작성

# query string
request_bbox = requests.get(bbox_url, params=bbox_params)

json_bbox = request_bbox.json()
print(json_bbox['rooms'][0])                                          # rooms 키 밑의 리스트로 구성되어 있음!
                                                  


{'is_favorited': None, 'seq': 16172100, 'id': '5fade41ac5b1a6787838014a', 'user_id': 'whrodzg@hanmail.net', 'status': 1, 'deleted': False, 'name': '조경연', 'title': '신도림 NO.1 아파트뷰 1.5룸 7평이상 전속물건 말이필요없어요', 'room_type': 0, 'location': [126.8967870879152, 37.5053309985632], 'random_location': [126.89685162265677, 37.50526500037477], 'complex_name': None, 'premium_badge': None, 'hash_tags': ['분리형', '주차', '풀옵션', '엘리베이터', '전세자금대출', '보안/안전'], 'room_type_str': '원룸', 'room_desc': '원룸 | 4층 | 23.14m² | 관리비 8만', 'img_url': 'http://d1774jszgerdmk.cloudfront.net/512/08d8ecb2-531f-4786-8b67-ba1670f0bc1b', 'img_urls': ['http://d1774jszgerdmk.cloudfront.net/512/08d8ecb2-531f-4786-8b67-ba1670f0bc1b', 'http://d1774jszgerdmk.cloudfront.net/512/ea1d2520-1ee5-4867-9646-e17739ba80f7', 'http://d1774jszgerdmk.cloudfront.net/512/e12f97ad-52f5-4df9-a48a-7eda4d6a14ee', 'http://d1774jszgerdmk.cloudfront.net/512/9818b9e2-ce9d-49ab-985b-8d3167d46ab0', 'http://d1774jszgerdmk.cloudfront.net/512/34579959-bf89-483b-82ca-0

In [55]:
# bbox JSON 테스트 1  - 필드 출력
bbox_field = list()
for i in json_bbox['rooms'][0]:
  bbox_field.append(i)
print(bbox_field)

['is_favorited', 'seq', 'id', 'user_id', 'status', 'deleted', 'name', 'title', 'room_type', 'location', 'random_location', 'complex_name', 'premium_badge', 'hash_tags', 'room_type_str', 'room_desc', 'img_url', 'img_urls', 'is_pano', 'price_title', 'selling_type', 'is_confirm', 'confirm_type', 'confirm_date_str', 'is_quick', 'is_messenger_actived', 'is_extend_ui', 'is_new_construction']


In [60]:
# bbox JSON 테스트 2  - id 출력
bbox_rooms = json_bbox['rooms']
# print(bbox_rooms[0]['id'])
for room in bbox_rooms:
  print("id: ", room['id'])

5fade41ac5b1a6787838014a
id:  5fade41ac5b1a6787838014a
id:  5fa203285a844f7cf90eab5e
id:  5fa2032d63960671c1dcbaf3
id:  5f8e401dad6c533f00bb94fb
id:  5f70a880cd818b5413169eee
id:  5fade4112f7a697b088a15c9
id:  5fade41f4a2fec7b08fc6e06
id:  5fade421cf88e805823f5d80
id:  5fade423f4668d317131a16d
id:  5fade416333c044a8d4285cb
id:  5f9000ff3449e31340dc8e33
id:  5fade4299cff1e35037f5c7b
id:  5fa20321d1807871c1e36248
id:  5fade418b67101317135e0ac
id:  5fade41d91e5020c8a4b8338
id:  5fa8ac936ce1fc0a4105b863
id:  5faa47366b8365322483b09f
id:  5f9e17c8e22d33493b969121
id:  5f979f7a89879048e05e8588
id:  5f9b8bd20fa42b12198e5365
id:  5faa00c1156c8e4c48a66651
id:  5fa61d6ff392723c71a47069
id:  5fa38d2404fb0c0afc1af1b6
id:  5f9cfdb7815d4239a95f3c2f


In [64]:
'''
  3. 분리된 bbox에서 얻은 방 정보에 대해 detail2 요청을 보내고 이후 얻어지는 정보를 파싱, 매물 데이터를 얻는다.
  api_version=3.0.1&call_type=web&room_id=5fa8a2447da6c80411ae0c1f&use_map=kakao&version=1

  url: https://www.dabangapp.com/api/3/room/detail2?
  params:
    'api_version' : '3.0.1'
    'call_type' : 'web'
    'room_id' : '5fa8a2447da6c80411ae0c1f'
    'use_map' : 'kakao'
    'version' : 1
'''
detail_url = 'https://www.dabangapp.com/api/3/room/detail2?'

# param: room_id 변수
idx_room = 0
var_id = bbox_rooms[idx_room]['id']

# params:
detail_params = {'api_version':'3.0.1', 'call_type':'web', 'usemap': 'kakao', 'version':1, 'room_id': var_id}

# query string
request_detail = requests.get(detail_url, params=detail_params)
json_detail = request_detail.json()
print(json_detail)


{'is_messenger_sender_agented': False, 'ios_score_height': 542, 'agent': {'id': '5b18e8ba7eebe55ad0e5cfc3', 'email': 'nms456@hanmail.net', 'name': '드림공인중개사사무소', 'facename': '노명식', 'address': '서울특별시 관악구 남부순환로144길 66 1층 101호(신림동,하늘명가)', 'location': [126.91312245859, 37.4783833066279], 'reg_id': '11650-2018-00185', 'business_id': '112-79-00162', 'saved_time': '2018-06-07T08:11:37.735Z', 'active': True, 'is_primary': True, 'user_name': '노명식', 'user_phone': '01057357777', 'user_safe_phone': '050432305796', 'user_tel': None, 'users_idx': 1733199, 'sub_emails': ['whrodzg@hanmail.net', 'jby981004@nate.com', 'flyhigh2030s@gmail.com', 'qkrwlsdn88@naver.com', 'hdskoko@naver.com', '8514021@naver.com', 'cjstkwnsxod1@naver.com', 'gmlwls11@naver.com', 'dnjrkfm22@naver.com', 'morning2@naver.com', 'scqudxo@naver.com', 'lsd4845@nate.com', 'yhw0305@naver.com', '-chohee-@hanmail.net'], 'is_premium': False, 'review_best_count': 58, 'review_count': 62, 'is_messenger_actived': False, 'profile_url': 'https://

In [70]:
# detail JSON 테스트 1  - 필드 출력
detail_field = list()
for i in json_detail:
  detail_field.append(i)
print(detail_field)

['is_messenger_sender_agented', 'ios_score_height', 'agent', 'is_consultation_actived', 'is_consultation_receipted', 'detail_shop', 'bubble_image', 'is_consultation_sender_agented', 'room', 'is_messenger_actived', 'score', 'review', 'is_messenger_receipted', 'contact', 'other_rooms', 'shinhanbank', 'messenger_send_alimtalk_contents', 'user', 'messenger_bubble_contents']


In [75]:
# detail JSON 테스트 2 - 방 상세 정보 출력(room_info)
room_info = json_detail['room']
for i in room_detail:
  print(i,": ", room_detail[i])

is_favorited :  None
favorited_count :  0
seq :  16172100
id :  5fade41ac5b1a6787838014a
user_id :  whrodzg@hanmail.net
status :  1
title :  신도림 NO.1 아파트뷰 1.5룸 7평이상 전속물건 말이필요없어요
memo :   !!!!압도적인 방 컨디션과 압도적인 가격이니 고민 하지마시고  얼른문의주세요!!!!
? 교통 및 위치
: 2,호선 신도림역 도보 5분!!

? 가격
: 반전세 보증금 1억2천이상 / 10(보증금 협의)
전세ㄷh출도가능
? 관ㄹ비
: 8만 전기 가스 수도 별도  
: 개별 , 인터넷 tv 올포함

? 주차시설
: 0만원 (협의) 

? 보안
: 바로앞 4차선 대로변, 지하철 역세권.
: 건물 내외부 CCTV, 1층 및 현관 도어락 설치.

? 편의시설
: 도보1분이내 편의점,식당,카페,각종병원.
: 도보10분이내 타임스퀘어,이마트,백화점,은행 등
: 테크노마트,신도림먹자골목,디큐브시티 
- 지하철5분 1.2호선 신도림역

? 특이사항
- 꿀 매물~!,신축 실7평이상 역대급 물건입니다.
- 보증금조절로 월세 얼마든지 맞출 수 있습니다.
- 전/월세 모두 가능합니다.
private_memo :  None
room_type :  0
maintenance_option :  0
room_size :  23.14
provision_size :  23.14
pano :  None
moving_date :  2020.12.15
card :  False
photos :  ['08d8ecb2-531f-4786-8b67-ba1670f0bc1b', 'ea1d2520-1ee5-4867-9646-e17739ba80f7', 'e12f97ad-52f5-4df9-a48a-7eda4d6a14ee', '9818b9e2-ce9d-49ab-985b-8d3167d46ab0', '34579959-bf89-483b-82ca-01642b0ef58e', '5c2f6637-a5e